In [21]:
# !pip install -U langchain-openai

In [36]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

class ContactOpenAIScraping:

    def __init__(self):
        self.api_key = ''
        self.llm = ChatOpenAI(
            model="gpt-3.5-turbo-1106",
            temperature=.9,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            api_key = self.api_key,
        )
        messages = [
            (
                "system", """You are an agent tasked with extracting contact information (emails, phone numbers, fax numbers, WhatsApp numbers, and addresses) from a contact page text. 
            
                => Ensure that the extracted information is valid. Do not include generic placeholders such as '123-456-7890', '000-000-0000', or 'info@example.com'.
                => Make sure the value of the key "phones" contains all valid phone numbers, fax numbers, and WhatsApp numbers.
                => Ensure the value of the key "addresses" contains valid, identifiable addresses.
                => Ensure you identify and extract the relevant information accurately.
                """
            ),
            (
                "human", """Extract the contact information (emails, phone numbers, and addresses) and format it into a JSON object from the following text. Make sure not to create any additional keys—only 'emails' and 'phones' and 'addresses'. Ensure the value of the key 'phones' contains all phone, fax, and WhatsApp numbers from the contact page text."""
            ),
            (
                "human","{contact_page_text}"
            )
            ]
        
        self.json_llm = self.llm.bind(response_format={"type": "json_object"})
        self.template = ChatPromptTemplate.from_messages(messages)
        self.chain = self.template | self.json_llm
        
    def predict(self, input_contact_page_text):
        ai_msg = self.chain.invoke({'contact_page_text':input_contact_page_text})
        results = {}
        results['content'] = json.loads(ai_msg.content)
        results['completion_tokens'] = ai_msg.response_metadata['token_usage']['completion_tokens']
        results['prompt_tokens'] = ai_msg.response_metadata['token_usage']['prompt_tokens']
        results['total_tokens'] = ai_msg.response_metadata['token_usage']['total_tokens']
        results['model_name'] = ai_msg.response_metadata['model_name']
        return results

In [26]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

api_key = ''
llm = ChatOpenAI(
            model="gpt-3.5-turbo-1106",
            temperature=.9,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            api_key = api_key,
        )

messages = [
    (
        "system",
        "You are a helpful {{translator}}. Translate the user sentence to French. with a json format, keys are 'English' and 'French'",
    ),
    ("human", '{sentence}'),
]
template = ChatPromptTemplate.from_messages(messages)
chain = template | llm | JsonOutputParser()
result = chain.invoke({'sentence':"I love programming."})
result

{'English': 'I love programming.', 'French': "J'adore la programmation."}

In [16]:
json_llm = llm.bind(response_format={"type": "json_object"})
ai_msg = json_llm.invoke(
    "Return a JSON object with key 'random_ints' and a value of 10 random ints in [0-99]"
)
ai_msg

AIMessage(content='{\n  "random_ints": [45, 78, 23, 11, 66, 89, 32, 55, 77, 19]\n}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 32, 'total_tokens': 70, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_0338b7694d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0dd2da47-c289-40ce-ba24-182952e8898e-0', usage_metadata={'input_tokens': 32, 'output_tokens': 38, 'total_tokens': 70})

In [24]:
ai_msg.response_metadata['model_name']

'gpt-3.5-turbo-1106'

In [28]:
html_code = """
<div class="styles_middle__dytqG">mouhsine abdelghaffour : +212 682103381 <div class="styles_container_col__r1Hsm"><div class="styles_container_img__KTxJW"><img alt="Logo Jump White" loading="lazy" width="135" height="48" decoding="async" data-nimg="1" srcset="https://join-jump.cdn.prismic.io/join-jump/Zh5pfEaI3ufuUOGq_jump_white.svg?auto=compress%2Cformat&amp;fit=max&amp;w=256 1x, https://join-jump.cdn.prismic.io/join-jump/Zh5pfEaI3ufuUOGq_jump_white.svg?auto=compress%2Cformat&amp;fit=max&amp;w=384 2x" src="https://join-jump.cdn.prismic.io/join-jump/Zh5pfEaI3ufuUOGq_jump_white.svg?auto=compress%2Cformat&amp;fit=max&amp;w=384" style="color: transparent;"></div><p>Les avantages des salariés, sans sacrifier votre liberté. Lâchez prise, on s’occupe vraiment de tout.</p><div class="styles_container_phone__qAIt9"><svg viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M20.2691 3.26051L16.7366 2.44676C15.9266 2.25926 15.1016 2.67926 14.7753 3.44426L13.1441 7.25051C12.8441 7.94801 13.0466 8.77301 13.6353 9.25301L15.1353 10.4793C14.0628 12.4443 12.4428 14.0643 10.4778 15.133L9.25159 13.633C8.77159 13.0443 7.94659 12.8418 7.24909 13.1418L3.43909 14.773C2.67409 15.0993 2.25784 15.9243 2.44159 16.7343L3.25534 20.2668C3.43534 21.0505 4.12534 21.598 4.92784 21.598C14.1116 21.6018 21.6003 14.1655 21.6003 4.93301C21.6003 4.13051 21.0528 3.44051 20.2691 3.26051ZM4.99909 19.8018L4.21534 16.4005L7.89784 14.8218L9.98659 17.3755C13.6916 15.6355 15.6341 13.7005 17.3741 9.98801L14.8203 7.89926L16.3991 4.21676L19.8003 5.00051C19.7628 13.1605 13.1591 19.7643 4.99909 19.8018Z"></path></svg><a rel="noreferrer" href="tel:+33176403201">+33 1 76 40 32 01</a></div><div class="styles_container_mail__4vqgc"><svg viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M19.8004 4.80078H4.20039C3.20627 4.80078 2.40039 5.60666 2.40039 6.60078V17.4008C2.40039 18.3949 3.20627 19.2008 4.20039 19.2008H19.8004C20.7945 19.2008 21.6004 18.3949 21.6004 17.4008V6.60078C21.6004 5.60666 20.7945 4.80078 19.8004 4.80078ZM19.8004 6.60078V8.13097C18.9596 8.81568 17.6191 9.88038 14.7534 12.1243C14.1218 12.6211 12.8708 13.8145 12.0004 13.8006C11.1301 13.8147 9.87868 12.6209 9.2474 12.1243C6.38214 9.88072 5.04133 8.81579 4.20039 8.13097V6.60078H19.8004ZM4.20039 17.4008V10.4407C5.05967 11.1251 6.27823 12.0855 8.13557 13.5399C8.9552 14.1851 10.3906 15.6094 12.0004 15.6007C13.6023 15.6094 15.0195 14.2058 15.8649 13.5402C17.7222 12.0859 18.9411 11.1252 19.8004 10.4407V17.4008H4.20039Z"></path></svg><a target="" rel="noreferrer" href="mailto:hello@join-jump.com">hello@join-jump.com</a></div><div class="styles_container_lang__QoIMJ "><div class="styles_lang__dJO1Q"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 513 342"><path fill="#FFF" d="M0 0h513v342H0z"></path><path fill="#0052B4" d="M0 0h171v342H0z"></path><path fill="#D80027" d="M342 0h171v342H342z"></path></svg><span>Français</span><svg viewBox="0 0 24 24"><path d="M15.443 12.3197L9.93797 17.8697C9.76172 18.0459 9.47672 18.0459 9.30047 17.8697L8.55797 17.1272C8.38172 16.9509 8.38172 16.6659 8.55797 16.4897L13.0017 12.0009L8.55797 7.51219C8.38172 7.33594 8.38172 7.05094 8.55797 6.87469L9.30047 6.13219C9.47672 5.95594 9.76172 5.95594 9.93797 6.13219L15.443 11.6822C15.6192 11.8584 15.6192 12.1434 15.443 12.3197Z"></path></svg></div><div class="styles_dropdown__P85js"><a href="/"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 513 342"><path fill="#FFF" d="M0 0h513v342H0z"></path><path fill="#0052B4" d="M0 0h171v342H0z"></path><path fill="#D80027" d="M342 0h171v342H342z"></path></svg><span>Français</span></a><a href="/en"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 513 342"><g fill="#FFF"><path d="M0 0h513v341.3H0V0z"></path><path d="M311.7 230 513 341.3v-31.5L369.3 230h-57.6zM200.3 111.3 0 0v31.5l143.7 79.8h56.6z"></path></g><path d="M393.8 230 513 295.7V230H393.8zm-82.1 0L513 341.3v-31.5L369.3 230h-57.6zm146.9 111.3-147-81.7v81.7h147zM90.3 230 0 280.2V230h90.3zm110 14.2v97.2H25.5l174.8-97.2zm-82.1-132.9L0 45.6v65.7h118.2zm82.1 0L0 0v31.5l143.7 79.8h56.6zM53.4 0l147 81.7V0h-147zm368.3 111.3L513 61.1v50.2h-91.3zm-110-14.2V0h174.9L311.7 97.1z" fill="#0052B4"></path><g fill="#D80027"><path d="M288 0h-64v138.7H0v64h224v138.7h64V202.7h224v-64H288V0z"></path><path d="M311.7 230 513 341.3v-31.5L369.3 230h-57.6zm-168 0L0 309.9v31.5L200.3 230h-56.6zm56.6-118.7L0 0v31.5l143.7 79.8h56.6zm168 0L513 31.5V0L311.7 111.3h56.6z"></path></g></svg><span>Anglais</span></a></div></div><div class="styles_container_sn__oB2Bs"><a target="_blank" rel="noreferrer" href="https://fr.linkedin.com/company/join-jump"><img alt="linkedin" loading="lazy" width="21" height="21" decoding="async" data-nimg="1" srcset="https://join-jump.cdn.prismic.io/join-jump/ZiZry_Pdc1huKrUn_linkedin.svg?auto=compress%2Cformat&amp;fit=max&amp;w=32 1x, https://join-jump.cdn.prismic.io/join-jump/ZiZry_Pdc1huKrUn_linkedin.svg?auto=compress%2Cformat&amp;fit=max&amp;w=48 2x" src="https://join-jump.cdn.prismic.io/join-jump/ZiZry_Pdc1huKrUn_linkedin.svg?auto=compress%2Cformat&amp;fit=max&amp;w=48" style="color: transparent;"></a><a target="_blank" rel="noreferrer" href="https://www.facebook.com/JoinJumpFrance/"><img alt="facebook" loading="lazy" width="21" height="21" decoding="async" data-nimg="1" srcset="https://join-jump.cdn.prismic.io/join-jump/ZiZryfPdc1huKrUl_facebook-square.svg?auto=compress%2Cformat&amp;fit=max&amp;w=32 1x, https://join-jump.cdn.prismic.io/join-jump/ZiZryfPdc1huKrUl_facebook-square.svg?auto=compress%2Cformat&amp;fit=max&amp;w=48 2x" src="https://join-jump.cdn.prismic.io/join-jump/ZiZryfPdc1huKrUl_facebook-square.svg?auto=compress%2Cformat&amp;fit=max&amp;w=48" style="color: transparent;"></a><a target="_blank" rel="noreferrer" href="https://www.instagram.com/joinjump_fr/"><img alt="instagram" loading="lazy" width="21" height="21" decoding="async" data-nimg="1" srcset="https://join-jump.cdn.prismic.io/join-jump/ZiZryvPdc1huKrUm_instagram-square.svg?auto=compress%2Cformat&amp;fit=max&amp;w=32 1x, https://join-jump.cdn.prismic.io/join-jump/ZiZryvPdc1huKrUm_instagram-square.svg?auto=compress%2Cformat&amp;fit=max&amp;w=48 2x" src="https://join-jump.cdn.prismic.io/join-jump/ZiZryvPdc1huKrUm_instagram-square.svg?auto=compress%2Cformat&amp;fit=max&amp;w=48" style="color: transparent;"></a></div></div><div data-slice-type="footer_links" data-slice-variation="default" class="styles_container_col__593Df"><div class="styles_title__T5R_8"><h4>En savoir plus</h4></div><ul><li><a href="/simulation-salaire-portage-salarial">Simulation portage salarial</a><div class="styles_tag__ItWua"></div></li><li><a href="/conseils/portage-salarial">Guides portage salarial</a><div class="styles_tag__ItWua"></div></li><li><a href="/conseils/freelance">Guides freelance</a><div class="styles_tag__ItWua"></div></li><li><a target="_blank" rel="noreferrer" href="https://comparateur.join-jump.com/">Comparateur de statuts</a><div class="styles_tag__ItWua"></div></li><li><a href="/business">Entreprises</a><div class="styles_tag__ItWua">New</div></li><li><a href="/tarifs">Tarifs</a><div class="styles_tag__ItWua"></div></li><li><a href="/programme-de-parrainage">Programme de parrainage</a><div class="styles_tag__ItWua">New</div></li></ul></div><div data-slice-type="footer_links" data-slice-variation="default" class="styles_container_col__593Df"><div class="styles_title__T5R_8"><h4>Ressources</h4></div><ul><li><a href="/actu">Actualités</a><div class="styles_tag__ItWua"></div></li><li><a href="/webinars">Webinars</a><div class="styles_tag__ItWua"></div></li><li><a href="/livres-blancs">Livres blancs</a><div class="styles_tag__ItWua"></div></li><li><a href="/avis-clients">Avis sur Jump</a><div class="styles_tag__ItWua"></div></li><li><a href="/conseils/glossaire">Glossaire</a><div class="styles_tag__ItWua"></div></li><li><a href="/faq">FAQ</a><div class="styles_tag__ItWua"></div></li><li><a href="/plan-de-site">Plan de site</a><div class="styles_tag__ItWua"></div></li></ul></div><div data-slice-type="footer_links" data-slice-variation="default" class="styles_container_col__593Df"><div class="styles_title__T5R_8"><h4>Nous contacter</h4></div><ul><li><a href="/a-propos">À propos</a><div class="styles_tag__ItWua"></div></li><li><a href="/carriere">Nous rejoindre</a><div class="styles_tag__ItWua"></div></li><li><a href="/contact">Contacter Jump</a><div class="styles_tag__ItWua"></div></li><li><a href="/villes">Où nous trouver ?</a><div class="styles_tag__ItWua"></div></li><li><a href="/presse">Presse</a><div class="styles_tag__ItWua"></div></li></ul></div></div>
"""

In [38]:
ContactOpenAIScraping().predict(html_code)

{'content': {'emails': ['hello@join-jump.com'],
  'phones': ['+212 682103381', '+33 1 76 40 32 01'],
  'addresses': []},
 'completion_tokens': 44,
 'prompt_tokens': 3881,
 'total_tokens': 3925,
 'model_name': 'gpt-3.5-turbo-1106'}